In [5]:
from river.drift import ADWIN
from river import ensemble
from river import tree
from river import datasets
from river import metrics
from river import anomaly
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd



# Crea instancias de los algoritmos y métricas
adwin = ADWIN()
model = tree.HoeffdingTreeClassifier(grace_period=10)
metric_acc = metrics.Accuracy()
metric_f1 = metrics.F1()

# Inicializa el detector de outliers
#isolation_forest = anomaly.IsolationForest()
isolation_forest = IsolationForest(contamination=0.05)

# Carga el conjunto de datos
data = datasets.CreditCard()

#plt.plot(data['Level'])

# Inicializa una variable para almacenar las columnas
columnas = set()

# Itera a través de los datos y registra las columnas
for x, _ in data:
    columnas.update(x.keys())

# Muestra las columnas
print("Columnas del conjunto de datos:")
for columna in columnas:
    print(columna)

# Inicializa variables para el seguimiento
n_samples = 0
change_detected = False


# Itera a través de los datos en tiempo real
for x, y in data:
    n_samples += 1

    # Realiza la detección de cambios
    if n_samples <= 200000:
        # Realiza el entrenamiento inicial con las primeras 50,000 muestras
        y_pred = model.predict_one(x)
        model.learn_one(x, y)
    else:
        if adwin.update(y):
            change_detected = True
            print(f"Change detected at sample {n_samples}")

        # Actualiza el clasificador de ensamble con los nuevos datos
        y_pred = model.predict_one(x)
        model.learn_one(x, y)

        # Evalúa el rendimiento
        metric_acc.update(y, y_pred)
        metric_f1.update(y, y_pred)

# Muestra las métricas finales
print(f"Number of samples: {n_samples}")
print("Accuracy:", metric_acc.get())
print("F1 Score:", metric_f1.get())
